# Problema del commesso viaggiatore

## Obiettivo e Prerequisiti

L'obiettivo del problema è quello di trovare il percorso più breve possibile che permetta al commesso di visitare ogni città una volta sola, ritornando alla città di partenza.

In questa esercitazione si farà uso della libreria dedicata all'ottimizzazione **Gurobi** (per cui è necessaria una licenza, free per uso accademico)

## Descrizione del problema
Definito un elenco di città e le distanze tra ogni coppia di esse, si vuole individuare il percorso più breve possibile che passi per ciascuna città una volta sola e ritorni a quella di partenza.

Nella presente esercitazione si assumerà che la distanza per andare dalla città $i$ alla città $j$ sia la stessa di quella per andare dalla città $j$ alla città $i$, questo tipo di problema del commesso viaggiatore è noto anche come problema simmetrico del commesso viaggiatore.

## Approccio risolutivo

La programmazione matematica rappresenta un approccio dichiarativo in cui viene definito un modello di ottimizzazione che cattura gli aspetti chiave di un problema decisionale complesso.

Un modello di ottimizzazione ha cinque componenti:

* Insiemi e indici
* Parametri
* Variabili decisionali
* Funzione/i obiettivo
* Vincoli

## Formulazione del modello

### Insiemi e indici
$i, j \in Capitali $: indici e insieme delle capitali degli USA.

$\text{Coppie}= \{(i,j) \in Capitali \times Capitali \}$: Insieme di coppie di capitali.

$S \subset Capitali$: Sottoinsieme dell'insieme di capitali degli USA.

$G = (Capitali, Coppie)$: Un grafo in cui l'insieme $Capitali$ definisce un insieme di nodi e $Coppie$ definisce un insieme di archi. 

### Parametri 

$d_{i, j} \in \mathbb{R}^+$: Distanza tra capitale $i$ e capitale $j$, per tutte le $(i, j) \in Coppie$. 

Occore notare che la distanza dalla capitale $i$ alla capitale $j$ è uguale alla distanza dalla capitale $j$ alla capitale $i$, cioè $d_{i, j} = d_{j, i} $. Per questo motivo, questa tipologia di problema è anche chiamato problema del commesso viaggiatore simmetrico.

### Variabili decisionali
$x_{i, j} \in \{0, 1\}$: Variabile è uguale ad 1, se si decide di connettere la città $i$ alla città $j$. Altrimenti la variabile decisionale è posta uguale a 0.

### Funzione obiettivo
- **Percorso più breve**. Minimizzazione della distanza totale di un percorso. Un percorso è rappresentato da una sequenza di capitali in cui il commesso visita ogni città solo una volta e torna alla capitale di partenza.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Coppie}}d_{i,j} \cdot x_{i,j}
\end{equation}

### Vincoli 
- **Vincoli di simmetria**. Per ogni arco $(i,j)$, ci si assicura che le capitali $i$ e $j$ siano collegate, se la prima viene visitata immediatamente prima o dopo aver visitato la seconda.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Coppie
\end{equation}

- **Entrare e uscire da una capitale**. Per ogni capitale $i$, ci si assicura che questa sia collegata ad altre due città.

\begin{equation}
\sum_{(i,j) \in \text{Coppie}}x_{i,j} = 2 \quad \forall  i \in Capitali
\end{equation}

- **Eliminazione di loop**. Questo vincolo assicura che per qualsiasi sottoinsieme di città $S$ dell'insieme di $Capitali$, non vi sia alcun ciclo. In altre parole, non esiste un percorso che visiti tutte le città del sottoinsieme e torni alla città di origine.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Capitali
\end{equation}

## Implementazione

In questa esercitazione sono state utilizzate le seguenti librerie:
* **folium**: per visualizzare la mappa finale.
* **gurobipy**: mette a disposizione gli algoritmi di Gurobi per risolvere i modelli.


### Lettura dei dati
I nomi delle capitali e le loro coordinate sono lette da un file json creato appositamente.

In [1]:
import json

# Lettura del file json contenete le capitali e le loro coordinate
capitals_json = json.load(open('capitalsUSA.json'))
capitals = []
coordinates = {}
for state in capitals_json:
    if state not in ['AK', 'HI']: #eliminazione di Alaska e Hawaii per evitare di passare nel mare
      capital = capitals_json[state]['capital']
      capitals.append(capital)
      coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

### Calcolo delle distanze
La seguente funzione calcola la distanza per ogni coppia di capitali. Dal momento che si sta risolvendo il problema del commesso viaggiatore simmetrico, si utilizza la *combination* delle città.

In [2]:
import math
from itertools import combinations

# Calcolo delle distanze a coppie

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

### Modellazione
Si definisce il modello per il problema, definendo variabili decisionali, vincoli e funzione obiettivo. Poiché si considera il problema del commesso viaggiatore simmetrico, è possibile renderlo più efficiente definendo x[j,i] = x[i,j].

In [3]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

# Variabili: la città 'i' è adiacente alla città 'j'?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Direzione simmetrica: copia dell'oggetto
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # archi in direzione opposta

# Vincoli: due archi incidenti a ciascuna città
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-24


### Callback
I vincoli per i sotto percorsi impediscono i cicli in un percorso. Poiché ne esiste un numero esponenziale non si vuole aggiungerli tutti al modello, quindi verrà utilizzata una funzione di callback per trovare vincoli violati e aggiungerli al modello come "vincoli pigri".

In [4]:
# Callback - utilizzo dei vincoli pigri per eliminare i sotto percorsi

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # creazione di una lista di archi selezionati nella soluzione
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # Identificazione del ciclo più corto nella lista degli archi
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # aggiunta del vincolo di eliminazione dei sotto percorsi per ogni coppia di città nel sotto percorso
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Data una lista di tuple di archi, identificare il percorso più breve

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy
    while unvisited:  # true se la lista non è vuota
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nuovo percorso breve
    return cycle

## Risoluzione del modello

In [5]:
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 48 rows, 1128 columns and 2256 nonzeros
Model fingerprint: 0x63641a38
Variable types: 0 continuous, 1128 integer (1128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 48 rows, 1128 columns, 2256 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 1.611858e+02, 72 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  161.18576    0   12          -  161.18576      -     -    0s
     0     0  163.11017    0   18          -  163.11017      -     -    0s
   

## Analisi

Si identifica la soluzione ottima del problema del commesso viaggiatore e si verifica che il percorso ottimale passi per tutte le città e ritorni alla città di origine.

In [6]:
vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

Il percorso ottimale è visualizzato sulla mappa.

In [7]:
import folium

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

## Conclusioni

Il problema del commesso viaggiatore è uno dei problemi di ottimizzazione più diffusi, è molto facile da spiegare pur essendo complesso da risolvere.

In questa esercitazione si è mostrato come formulare il problema del commesso viaggiatore simmetrico considerando le capitali degli Stati Uniti d'America e si è compreso come eliminare dinamicamente i sotto percorsi utilizzando i "vincoli pigri".

Effettuando un paragone con l'esercitazione svolta considerando i capoluoghi italiani (numericamente inferiori rispetto alle capitali statunitensi) si può notare che il metodo del simplesso, sul quale si basa il modello, effettua 2675 iterazioni (contro le 45 del caso dei capoluoghi d'Italia) per ottenere la soluzione ottima. 